In [18]:
#read csv to df and inspect for cleaning
import pandas as pd 
import numpy as np

df = pd.read_csv('burrito_shack_digital_performance.csv')

#inspect dataframe
#print(df.info())

#check for missing values
print(df.isnull().sum())

date                     0
location_id              0
city                     0
state                    0
region                   0
sessions                 0
page_views               0
bounce_rate             81
conversion_rate          0
online_orders            0
avg_order_value        161
revenue                  0
ad_spend_social        223
ad_spend_search          0
ad_spend_display         0
impressions_social       0
impressions_search       0
impressions_display      0
clicks_social            0
clicks_search            0
clicks_display           0
dtype: int64


In [20]:
# Clean 'date' column with multiple formats 

# 0) keep original for audit
df['date_raw'] = df['date'].astype(str).str.strip()

s = df['date_raw']

# 1) masks for each accepted format
mask_iso_dash   = s.str.match(r'^\d{4}-\d{1,2}-\d{1,2}$')  # YYYY-MM-DD
mask_us_slash   = s.str.match(r'^\d{1,2}/\d{1,2}/\d{4}$')  # MM/DD/YYYY
mask_iso_slash  = s.str.match(r'^\d{4}/\d{1,2}/\d{1,2}$')  # YYYY/MM/DD   <-- NEW

# 2) parse each bucket with explicit formats
parsed = pd.Series(pd.NaT, index=df.index, dtype='datetime64[ns]')
parsed.loc[mask_iso_dash]  = pd.to_datetime(s.loc[mask_iso_dash],  format='%Y-%m-%d', errors='coerce')
parsed.loc[mask_us_slash]  = pd.to_datetime(s.loc[mask_us_slash],  format='%m/%d/%Y', errors='coerce')
parsed.loc[mask_iso_slash] = pd.to_datetime(s.loc[mask_iso_slash], format='%Y/%m/%d', errors='coerce')  # <-- NEW

# 3) assign normalized date
df['date'] = parsed.dt.normalize()

# 4) audit
unknown_mask = ~(mask_iso_dash | mask_us_slash | mask_iso_slash)
counts = pd.Series({
    'YYYY-MM-DD' : int(mask_iso_dash.sum()),
    'MM/DD/YYYY' : int(mask_us_slash.sum()),
    'YYYY/MM/DD' : int(mask_iso_slash.sum()),
    'UNKNOWN'    : int(unknown_mask.sum()),
})
print("Rows by detected pattern:\n", counts)

print("\nUnknown examples:", df.loc[unknown_mask, 'date_raw'].head(15).tolist())
print("Failed-to-parse examples:", df.loc[df['date'].isna(), 'date_raw'].head(15).tolist())

# 5) (optional) bound check
lo, hi = pd.Timestamp('2019-01-01'), pd.Timestamp.today().normalize()
oob = df['date'].notna() & ~df['date'].between(lo, hi)
print(f"\nOut-of-range dates: {int(oob.sum())}")

# all clean, drop raw column
df.drop(columns=['date_raw'], inplace=True)
print("\nDate cleaning complete.\nRaw column dropped.")


Rows by detected pattern:
 YYYY-MM-DD    5000
MM/DD/YYYY       0
YYYY/MM/DD       0
UNKNOWN          0
dtype: int64

Unknown examples: []
Failed-to-parse examples: []

Out-of-range dates: 0

Date cleaning complete.
Raw column dropped.


In [ ]:
#FIX NULLS

#print(df.isnull().sum()) #bounce_rate (81), avg_order_value (161), ad_spend_social (223)

#tackle bounce_rate nulls by filling with average bounce rate by region
df['bounce_rate'] = df['bounce_rate'].fillna(
    df.groupby('region')['bounce_rate'].transform('median')
)

#check null counts in bounce_rate
print('bounce_rate nulls:', df['bounce_rate'].isnull().sum()) #0

#tackle avg_order_value nulls by filling with average order value by region
df['avg_order_value'] = df['avg_order_value'].fillna(
    df.groupby('region')['avg_order_value'].transform('median')
)
#check null counts in avg_order_value
print('avg_order_value nulls:', df['avg_order_value'].isnull().sum())

#review rows with ad_spend_social nulls
ad_spend_nulls = df[df['ad_spend_social'].isnull()]
print('Rows with ad_spend_social nulls:\n', ad_spend_nulls)

#missing ad_spend_social likely indicates no ad spend (or missing data), so flag nulls then fill nulls with 0
df['flag_ad_spend_missing'] = df['ad_spend_social'].isna()
df['ad_spend_social'] = df['ad_spend_social'].fillna(0)
print('ad_spend_social nulls:', df['ad_spend_social'].isnull().sum()) #0


bounce_rate nulls: 0
avg_order_value nulls: 0
Rows with ad_spend_social nulls:
            date location_id         city state region  sessions  page_views  \
13   2024-11-16      LA-001  Los Angeles    CA   West      1336        5782   
17   2024-11-20      LA-001  Los Angeles    CA   West      1156        3656   
172  2025-07-03      LA-001  Los Angeles    CA   West      1291        4366   
188  2025-07-28      LA-001  Los Angeles    CA   West      1228        4685   
199  2025-08-12      LA-001  Los Angeles    CA   West      1639        6031   
...         ...         ...          ...   ...    ...       ...         ...   
4897 2025-05-05     MIA-001        Miami    FL  South       691        2958   
4904 2025-05-14     MIA-001        Miami    FL  South       790        3195   
4907 2025-05-18     MIA-001        Miami    FL  South      1177        3538   
4971 2025-08-10     MIA-001        Miami    FL  South      1925        7863   
4998 2025-09-10     MIA-001        Miami    FL  Sou

In [33]:
#data validation on metrics
#bounce_rate should be between 0 and 1
invalid_bounce = ~df['bounce_rate'].between(0, 1)
print(f"Invalid bounce_rate entries: {int(invalid_bounce.sum())}")
#avg_order_value should be non-negative
invalid_aov = df['avg_order_value'] < 0
print(f"Invalid avg_order_value entries: {int(invalid_aov.sum())}")
#ad_spend_social should be non-negative
invalid_ad_spend = df['ad_spend_social'] < 0
print(f"Invalid ad_spend_social entries: {int(invalid_ad_spend.sum())}")
#conversion_rate should be between 0 and 1
invalid_conversion = ~df['conversion_rate'].between(0, 1)
print(f"Invalid conversion_rate entries: {int(invalid_conversion.sum())}")


Invalid bounce_rate entries: 0
Invalid avg_order_value entries: 0
Invalid ad_spend_social entries: 0
Invalid conversion_rate entries: 0


In [ ]:
#export cleaned dataframe to new csv, write to react public folder
df.to_csv('burrito-shack-analytics/public/burrito_shack_digital_performance_cleaned.csv', index=False)